In [2]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark.sql.functions import col, array_contains, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
# Connect to MongoDB
client = MongoClient('mongodb://mongodb:27017/')
db = client.Youtube_Database
collection = db.youtube_data_us

# Print the first document
print(collection.find_one())

# Close the connection
client.close()

{'_id': ObjectId('676ebf9db2412e2d5055f480'), 'video_id': '3C66w5Z0ixs', 'title': 'I ASKED HER TO BE MY GIRLFRIEND...', 'publishedAt': '2020-08-11T19:20:14Z', 'channelId': 'UCvtRTOMP2TqYqu51xNrqAzg', 'channelTitle': 'Brawadis', 'categoryId': 22, 'trending_date': '2020-08-12T00:00:00Z', 'tags': ['brawadis', 'prank', 'basketball', 'skits', 'ghost', 'funny videos', 'vlog', 'vlogging', 'NBA', 'browadis', 'challenges', 'bmw i8', 'faze rug', 'faze rug brother', 'mama rug and papa rug'], 'view_count': 1514614, 'likes': 156908, 'dislikes': 5855, 'comment_count': 35313, 'thumbnail_link': 'https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg', 'comments_disabled': False, 'ratings_disabled': False, 'description': 'SUBSCRIBE to BRAWADIS ▶ http://bit.ly/SubscribeToBrawadis\r\rFOLLOW ME ON SOCIAL\r▶ Twitter: https://twitter.com/Brawadis\r▶ Instagram: https://www.instagram.com/brawadis/\r▶ Snapchat: brawadis\r\rHi! I’m Brandon Awadis and I like to make dope vlogs, pranks, reactions, challenges and basketba

## Spark das empresas

In [4]:
spark_company = SparkSession.builder \
    .appName("company_database") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/Company_Database") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/Company_Database") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

## Spark dos paises youtube

In [5]:
spark_yt = SparkSession.builder \
    .appName("youtube_us") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/Youtube_Database") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/Youtube_Database") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

## Importanção dos dados de cada pais

In [6]:
df_us = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_us").load()
df_br = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_br").load()
df_ca = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_ca").load()
df_de = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_de").load()
df_fr = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_fr").load()
df_gb = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_gb").load()
df_in = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_in").load()
df_jp = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_jp").load()
df_kr = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_kr").load()
df_mx = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_mx").load()
df_ru = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Youtube_Database.youtube_data_ru").load()

## Importanção dos dados de empresa

In [7]:
df_dell = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.dell_data").load()
df_ibm = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.ibm_data").load()
df_intel = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.intel_data").load()
df_microsoft = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.microsoft_data").load()
df_nvidia = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.nvidia_data").load()
df_sony = spark_yt.read.format("mongo").option("uri", "mongodb://mongodb:27017/Company_Database.sony_data").load()

## Dicionario das empresas e seus nomes para posteriormente fazer a união dos dfs com a coluna company_name

In [8]:
dict_companies = {'companies': [
                 {'dataframe':df_dell, 'name': 'dell'},
                 {'dataframe':df_ibm, 'name': 'ibm'},
                 {'dataframe':df_intel, 'name': 'intel'},
                 {'dataframe':df_microsoft, 'name': 'microsoft'},
                 {'dataframe':df_nvidia, 'name': 'nvidia'},
                 {'dataframe':df_sony, 'name': 'sony'},  
                ]
            }

In [9]:
for company in dict_companies['companies']:
    company["dataframe"] = company["dataframe"].withColumn("company_name", lit(company["name"]))

In [12]:
df_all_companies = dict_companies['companies'][0]['dataframe'].union(dict_companies['companies'][1]['dataframe']).union(dict_companies['companies'][2]['dataframe']).union(dict_companies['companies'][3]['dataframe']).union(dict_companies['companies'][4]['dataframe']).union(dict_companies['companies'][5]['dataframe'])

In [13]:
#df_all_companies.show()

In [14]:
#df_all_companies.orderBy(col("company_name").desc()).show()

## União dos df de cada pais num só

In [15]:
df_yt = df_us.union(df_br).union(df_ca).union(df_de).union(df_fr).union(df_gb).union(df_in).union(df_jp).union(df_kr).union(df_mx).union(df_ru)

## Colunas existentes

In [16]:
df_yt.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- categoryId: integer (nullable = true)
 |-- channelId: string (nullable = true)
 |-- channelTitle: string (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- tags: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- title: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- view_count: integer (nullable = true)



In [17]:
df_all_companies.printSchema()

root
 |-- Adj Close: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- company_name: string (nullable = false)



## Quantidade de videos relacionados a x empresa

In [18]:
filtered_df_nvidia = df_yt.filter(
    (col('description').contains('nvidia')) |
    (col('title').contains('nvidia')) |
    (col('tags').contains('nvidia')) |  
    (col('channelTitle').contains('nvidia')) |
    (col('thumbnail_link').contains('nvidia'))
)

num_rows = filtered_df_nvidia.count()
print(f"Número de linhas que contêm 'nvidia': {num_rows}")

Número de linhas que contêm 'nvidia': 2568


In [19]:
filtered_df_ibm = df_yt.filter(
    (col('description').contains('ibm')) |
    (col('title').contains('ibm')) |
    (col('tags').contains('ibm')) |  
    (col('channelTitle').contains('ibm')) |
    (col('thumbnail_link').contains('ibm'))
)

num_rows = filtered_df_ibm.count()
print(f"Número de linhas que contêm 'ibm': {num_rows}")

Número de linhas que contêm 'ibm': 958


In [20]:
filtered_df_dell = df_yt.filter(
    (col('description').contains('dell')) |
    (col('title').contains('dell')) |
    (col('tags').contains('dell')) |  
    (col('channelTitle').contains('dell')) |
    (col('thumbnail_link').contains('dell'))
)

num_rows = filtered_df_dell.count()
print(f"Número de linhas que contêm 'dell': {num_rows}")

Número de linhas que contêm 'dell': 17685


In [21]:
filtered_df_intel = df_yt.filter(
    (col('description').contains('intel')) |
    (col('title').contains('intel')) |
    (col('tags').contains('intel')) |  
    (col('channelTitle').contains('intel')) |
    (col('thumbnail_link').contains('intel'))
)

num_rows = filtered_df_intel.count()
print(f"Número de linhas que contêm 'intel': {num_rows}")

Número de linhas que contêm 'intel': 17326


In [22]:
filtered_df_microsoft = df_yt.filter(
    (col('description').contains('microsoft')) |
    (col('title').contains('microsoft')) |
    (col('tags').contains('microsoft')) |  
    (col('channelTitle').contains('microsoft')) |
    (col('thumbnail_link').contains('microsoft'))
)

num_rows = filtered_df_microsoft.count()
print(f"Número de linhas que contêm 'microsoft': {num_rows}")

Número de linhas que contêm 'microsoft': 2958


In [ ]:
filtered_df_sony = df_yt.filter(
    (col('description').contains('sony')) |
    (col('title').contains('sony')) |
    (col('tags').contains('sony')) |  
    (col('channelTitle').contains('sony')) |
    (col('thumbnail_link').contains('sony'))
)

num_rows = filtered_df_sony.count()
print(f"Número de linhas que contêm 'sony': {num_rows}")

## União dos videos de cada empresa

In [ ]:
df_yt_companies = df_us.union(filtered_df_nvidia).union(filtered_df_ibm).union(filtered_df_dell).union(filtered_df_intel).union(filtered_df_microsoft).union(filtered_df_sony)

df_yt_companies - todos os videos das empresas a serem estudadas<br> 
df_all_companies - dados de todas as empreas a serem estudadas

## Estudo dos nulos

In [ ]:
type(df_yt_companies.na)

In [ ]:
df_yt_companies.na